# Install Requirements

In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from create_data_gb1 import create_xy
import tensorflow as tf
import keras
from keras.api.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.api.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

2024-07-08 01:10:06.339694: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 01:10:06.401183: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 01:10:06.565442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 01:10:06.715995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 01:10:06.725059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 01:10:06.979545: I tensorflow/core/platform/cpu_feature_guard.cc:

In [5]:
df = pd.read_excel("../data/gbd1_data.xlsx", header=2, usecols="B:K,N:R", nrows=250000)
f = open("../data/base_gbd1.txt")
base = f.readline()
f.close()

/home/tubtub/code/PE/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
mutants, fitness = create_xy(df, base)

In [7]:
def aa_identity(aa):
    aa_string = "ARNDCQEGHILKMFPSTWYV"
    for i in range(len(aa_string)):
        if aa == aa_string[i]:
            return i

In [8]:
mt = mutants[0]
def m_identity(mt):
    mi = np.zeros((20, len(mt)))
    for i in range(len(mt)):
        row = aa_identity(mt[i])
        mi[row][i] = 1
    return mi

mutant_identity = []
for mutant in mutants:
    mutant_identity.append(m_identity(mutant))
mutant_identity = np.array(mutant_identity)
print(mutant_identity.shape)
print(fitness.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(mutant_identity, fitness, test_size=0.2, shuffle=True, random_state=15)
print(f"{x_train.shape}\n{x_test.shape}\n{y_train.shape}\n{y_test.shape}")


In [ ]:
def model_1(x_train, y_train, x_test, y_test, epochs=20, batch_size=64):
    model = Sequential()
    model.add(keras.Input((20,56)))
    #model.add(Flatten())
    model.add(Conv1D(filters=128, kernel_size=12, activation='relu', use_bias=True, padding='same'))
    model.add(MaxPooling1D(2))    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', use_bias=True))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu', use_bias=True))
    model.add(Flatten())
    model.add(Dense(128, activation='relu')) #evt tilføj dropout istedet for så mange dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # Change to the appropriate number of classes for multi-class classification
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_test, y_test))
    return model, history

In [ ]:
model, history = model_1(x_train, y_train, x_test, y_test)